# Assigning Pathways

During NeuroMMSig v1.0, pathways were manually assigned to each edge. For v2.0, we would like to automate this process first with a rule based system, then a machine learning system for prioritizing curation before resorting to manual curation.

## Preamble

### Imports

In [1]:
import getpass
import itertools as itt
import os
import random
import sys
import time
from collections import defaultdict

import bio2bel_wikipathways
import hbp_knowledge
import pybel
from pybel.dsl import BaseAbundance, ListAbundance

### Environment

In [2]:
print(time.asctime())

Fri Aug 16 16:21:12 2019


In [3]:
print(sys.version)

3.7.3 (default, Mar 27 2019, 09:23:39) 
[Clang 10.0.0 (clang-1000.11.45.5)]


In [4]:
print(getpass.getuser())

cthoyt


In [5]:
pybel.get_version()

'0.13.3-dev'

In [6]:
print(hbp_knowledge.VERSION)

0.0.7


In [7]:
print(bio2bel_wikipathways.get_version())

0.2.4-dev


### Data

In [8]:
graph = hbp_knowledge.get_graph()
graph.summarize()

Human Brain Pharmacome Knowledge v0.0.7
Number of Nodes: 6023
Number of Edges: 21625
Number of Citations: 358
Number of Authors: 2012
Network Density: 5.96E-04
Number of Components: 31
Number of Warnings: 35


## Assigning Pathways

Generate mappings from a given database to HGNC gene identifiers.

In [9]:
wikipathways_manager = bio2bel_wikipathways.Manager()
wikipathways_manager.summarize()

{'pathways': 556, 'proteins': 6613}

In [10]:
pathway_to_symbols = {
    (pathway.wikipathways_id, pathway.name): {
        protein.hgnc_symbol
        for protein in pathway.proteins
    }
    for pathway in wikipathways_manager._query_pathway().all()
}

In [11]:
pathway_to_key = defaultdict(set)
double_annotated = defaultdict(lambda: defaultdict(list))

### Assigning HGNC-HGNC Edges

1. `If` the subject and object in an edge are both in a canonical pathway, then the edge gets assigned to the pathway.
2. `Else if` only one of the subject and the object in the edge have been assigned in the pathway:
  1. `If` the edge is an ontological edge, than add it to the pathway
  2. `If` there are other edges in the pathway mentioned in the same article, assign the edge to the pathway
  3. `Else` leave for manual curation
3. `Else if` neither of the nodes are assigned to the pathway, but both nodes are connected to nodes in the pathway by directed edges, assign both edge to the pathway as well as incident edges
4. `Else` the nodes don't get assigned to the pathway

In [12]:
c = 0

for u, v, k, d in graph.edges(keys=True, data=True):
    if not isinstance(u, BaseAbundance) or not isinstance(v, BaseAbundance):
        continue
    
    if u.namespace.lower() != 'hgnc' or v.namespace.lower() != 'hgnc':
        continue
    
    u_name, v_name = u.name, v.name

    for (pathway_id, pathway), symbols in pathway_to_symbols.items():
        if u_name in symbols and v_name in symbols:
            double_annotated['wikipathways', pathway_id, pathway][tuple(sorted([u_name, v_name]))].append((u, v, k, d))
            pathway_to_key['wikipathways', pathway_id, pathway].add(k)
            c += 1
            
print(f'Made {c} annotations')

Made 9423 annotations


### Assigning Chemical/Biological Process/Disese - HGNC edges

`If` an entity is related to a gene in a pathway, then that edge gets annotated to the pathway

In [13]:
c = 0

for u, v, k, d in graph.edges(keys=True, data=True):
    if not isinstance(u, BaseAbundance) or not isinstance(v, BaseAbundance):
        continue

    if u.namespace.lower() == 'hgnc' and v.namespace.lower() != 'hgnc':
        gene_name = u.name
        other_name = v.name
    elif u.namespace.lower() != 'hgnc' and v.namespace.lower() == 'hgnc':
        gene_name = v.name
        other_name = u.name
    else:
        continue

    for (pathway_id, pathway), symbols in pathway_to_symbols.items():
        if gene_name in symbols:
            double_annotated['wikipathways', pathway_id, pathway][tuple(sorted([gene_name, other_name]))].append((u, v, k, d))
            pathway_to_key['wikipathways', pathway_id, pathway].add(k)
            c += 1
            
print(f'Made {c} annotations')

Made 62318 annotations


### Assigning Complexes

If two or more members of a complex are in a pathway, then the whole complex and all of its partOf relationships will get assigned to that pathway.

In [14]:
c = 0
for node in graph:
    if not isinstance(node, ListAbundance):
        continue
    
    hgnc_count = sum(
        member.namespace.lower() == 'hgnc'
        for member in node.members
        if isinstance(member, BaseAbundance)
    )
    
    if 0 == hgnc_count:
        continue

    for (pathway_id, pathway), symbols in pathway_to_symbols.items():
        in_count = sum(
            member.name in symbols
            for member in node.members
            if isinstance(member, BaseAbundance) and member.namespace.lower() == 'hgnc'
        )
        
        do_it = (
            (1 == hgnc_count and 1 == in_count)  # Other stuff going on, lets do it
            or 2 <= in_count  # enough is going on
        
        )
        
        if not do_it:
            continue
        
        for u, v, k, d in graph.edges(node, keys=True, data=True):
            double_annotated['wikipathways', pathway_id, pathway][node].append((u, v, k, d))
            pathway_to_key['wikipathways', pathway_id, pathway].add(k)
            c += 1
            
print(f'Made {c} annotations')

Made 9398 annotations


### Print Results

In [15]:
with open('assignments.tsv', 'w') as file, open('assignments.rst', 'w') as log_file:
    print('database', 'pathway_id', 'pathway_name', 'key', 'bel', sep='\t', file=file)
    for (db, pathway_id, pathway), names_dict in double_annotated.items():
        title = f'{db}:{pathway_id} - {pathway}'
        print(title, file=log_file)
        print('=' * len(title), file=log_file)

        for node_key, keys_and_data in names_dict.items():
            print('', file=log_file)
            print(node_key, file=log_file)
            l = len(str(node_key))
            print('-' * l, file=log_file)
            for u, v, key, data in keys_and_data:
                print('-', key[:8], graph.edge_to_bel(u, v, data), file=log_file)
                print(db, pathway_id, pathway, key, graph.edge_to_bel(u, v, data), sep='\t', file=file)

        print('', file=log_file)

In [16]:
annotated_edge_keys = set(itt.chain.from_iterable(pathway_to_key.values()))
n_edges_annotated = len(annotated_edge_keys)

print(f'{n_edges_annotated} ({n_edges_annotated / graph.number_of_edges():.2%}) of {graph.number_of_edges()} edges were annotated')

8726 (40.35%) of 21625 edges were annotated


### Investigating what's Left

- Dealing with orthologs
- Reasoning over hierarchical relations (isA, partOf, hasMember)
- Protein complex membership for GO cellular components
- Checking protein families
- Annotation of GO cellular components to pathways
- Reactions - need to enrich with connections to biological processes in GO or annotate based on any enzymes that they interact with.

In [17]:
unannotated = [
    (u, v, k, d)
    for u, v, k, d in graph.edges(data=True, keys=True)
    if k not in annotated_edge_keys
]
len(unannotated)

12899

In [18]:
for u, v, k, d in random.sample(unannotated, 15):
    print(k[:8], graph.edge_to_bel(u, v, d))

67f89e8a a(CONSO:"paired helical filaments") positiveCorrelation a(CONSO:"3-3 dityrosine cross-linking")
0937f864 p(MGI:"chitinase-like 1") association p(MGI:"v-rel reticuloendotheliosis viral oncogene homolog A (avian)")
f5e79c99 bp(GO:"protein K48-linked ubiquitination") increases act(complex(GO:"proteasome complex"))
5d31e0e9 complex(p(HGNC:PFDN2), p(HGNC:PFDN2)) hasComponent p(HGNC:PFDN2)
a39436f6 p(FPLX:HSP90) hasVariant p(FPLX:HSP90, pmod(Ac))
cb80636b path(MESH:Aging) negativeCorrelation a(MESH:"Cerebrospinal Fluid")
19025c96 rxn(reactants(a(CHEBI:"D-glyceraldehyde 3-phosphate"), a(CHEBI:"dihydroxyacetone phosphate")), products(a(CHEBI:ATP))) hasReactant a(CHEBI:"D-glyceraldehyde 3-phosphate")
c7fd2d60 g(HGNC:C9orf72) association p(HGNC:RAB5A)
49e016e6 a(MESH:"Cellular Structures") association p(FPLX:"CCT_complex")
f8b989ab a(CHEBI:"chondroitin sulfate") decreases tloc(p(CONSO:"Tau isoform F (441 aa)"), fromLoc(GO:"extracellular region"), toLoc(GO:intracellular))
a45c193f bp(GO: